In [1]:
import requests
from lxml import etree
import pandas as pd
import json
import re
import time
import random 

反复查看请求与响应，发现每次POST请求Ajax之前，都会先发送一个Get请求来获取用户的Session（Cookie）信息。随后发送的Ajax请求会将此信息携带到cookie中，这样才会请求到正真的数据。

# 一页爬取为例

In [2]:
url='https://www.lagou.com/jobs/positionAjax.json?needAddtionalResult=false'

In [3]:
headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36',
        'Referer': 'https://www.lagou.com/jobs/list_python?labelWords=&fromSearch=true&suginput='
        }



In [4]:
#设置一个会话
session = requests.session()

In [5]:
session.get('https://www.lagou.com/jobs/list_java?labelWords=&fromSearch=true&suginput=&labelWords=hot',headers=headers)


<Response [200]>

In [6]:
data = {
'first':'true',
'pn':1,
'kd':'java'
}

In [8]:
#使用更新后的hsession请求Ajax json
rep = session.post('https://www.lagou.com/jobs/positionAjax.json?city=%E5%8C%97%E4%BA%AC&needAddtionalResult=false',headers=headers,data=data)

In [9]:
# 请求成功，接下来就是提取想要的信息。
print(rep.json())

{'success': True, 'msg': None, 'code': 0, 'content': {'showId': '49b4321c866f4cc0b6baabc084e30d5c', 'hrInfoMap': {'7101486': {'userId': 9429482, 'portrait': 'i/image2/M01/0E/AC/CgotOVyhgdKAWLwZAACHltqNgkc507.png', 'realName': '王玖滢', 'positionName': 'HRBP', 'phone': None, 'receiveEmail': None, 'userLevel': 'G1', 'canTalk': True}, '4770607': {'userId': 7349721, 'portrait': None, 'realName': 'PCGzhaopin', 'positionName': '招聘经理', 'phone': None, 'receiveEmail': None, 'userLevel': 'G1', 'canTalk': True}, '7021698': {'userId': 6155898, 'portrait': None, 'realName': 'zhaoxin', 'positionName': '招聘组', 'phone': None, 'receiveEmail': None, 'userLevel': 'G1', 'canTalk': True}, '7109510': {'userId': 6814513, 'portrait': 'i/image2/M01/96/50/CgoB5l2dVWyAY1uMAADgDTnea8Y16.jpeg', 'realName': '冯女士', 'positionName': 'HR', 'phone': None, 'receiveEmail': None, 'userLevel': 'G1', 'canTalk': True}, '7167974': {'userId': 16120742, 'portrait': 'i/image/M00/01/A2/CgqCHl6udlyAY_uRAA8i6hYS_XI805.jpg', 'realName': 

In [10]:
html=rep.json()

In [11]:
html

{'success': True,
 'msg': None,
 'code': 0,
 'content': {'showId': '49b4321c866f4cc0b6baabc084e30d5c',
  'hrInfoMap': {'7101486': {'userId': 9429482,
    'portrait': 'i/image2/M01/0E/AC/CgotOVyhgdKAWLwZAACHltqNgkc507.png',
    'realName': '王玖滢',
    'positionName': 'HRBP',
    'phone': None,
    'receiveEmail': None,
    'userLevel': 'G1',
    'canTalk': True},
   '4770607': {'userId': 7349721,
    'portrait': None,
    'realName': 'PCGzhaopin',
    'positionName': '招聘经理',
    'phone': None,
    'receiveEmail': None,
    'userLevel': 'G1',
    'canTalk': True},
   '7021698': {'userId': 6155898,
    'portrait': None,
    'realName': 'zhaoxin',
    'positionName': '招聘组',
    'phone': None,
    'receiveEmail': None,
    'userLevel': 'G1',
    'canTalk': True},
   '7109510': {'userId': 6814513,
    'portrait': 'i/image2/M01/96/50/CgoB5l2dVWyAY1uMAADgDTnea8Y16.jpeg',
    'realName': '冯女士',
    'positionName': 'HR',
    'phone': None,
    'receiveEmail': None,
    'userLevel': 'G1',
    'can

In [13]:
htmlnew=html['content']

In [14]:
type(htmlnew)

dict

In [15]:
htmlnew

{'showId': '2a376f0354224046a52cebfe80027886',
 'hrInfoMap': {'4770607': {'userId': 7349721,
   'portrait': None,
   'realName': 'PCGzhaopin',
   'positionName': '招聘经理',
   'phone': None,
   'receiveEmail': None,
   'userLevel': 'G1',
   'canTalk': True},
  '6641602': {'userId': 10123507,
   'portrait': 'i/image/M00/90/9C/CgpFT1sOeiKAUPudAAAvJ4YDv-Q077.png',
   'realName': '杨晓',
   'positionName': 'HRBP',
   'phone': None,
   'receiveEmail': None,
   'userLevel': 'G1',
   'canTalk': True},
  '6785071': {'userId': 9327565,
   'portrait': 'i/image2/M01/BA/91/CgotOVwY0gmAHD0AAAEqBzzYuNo388.jpg',
   'realName': '胡娜',
   'positionName': 'HR',
   'phone': None,
   'receiveEmail': None,
   'userLevel': 'G1',
   'canTalk': True},
  '7109510': {'userId': 6814513,
   'portrait': 'i/image2/M01/96/50/CgoB5l2dVWyAY1uMAADgDTnea8Y16.jpeg',
   'realName': '冯女士',
   'positionName': 'HR',
   'phone': None,
   'receiveEmail': None,
   'userLevel': 'G1',
   'canTalk': True},
  '7160231': {'userId': 558330

In [ ]:
#companyLabel_List.extend(em['companyLabelList'])

In [16]:
position=[]
company_name=[]
company_Size=[]
industry_Field=[]
finance_Stage=[]
companyLabel_List=[]
first_Type=[]
second_Type=[]
third_Type=[]
skill_Lables=[]
position_Lables=[]
industry_Lables=[]
create_Time=[]
cityn=[]
districtn=[]
business_Zones=[]
salaryn=[]
work_Year=[]
job_Nature=[]
educationn=[]
position_Advantage=[]
linestaionn=[]
latituden=[]
longituden=[]

for em in htmlnew['positionResult']['result']:
    position.append(em['positionName'])
    company_name.append(em['companyFullName'])
    company_Size.append(em['companySize'])
    industry_Field.append(em['industryField'])
    finance_Stage.append(em['financeStage'])
    companyLabel_List.append(em['companyLabelList'])
    first_Type.append(em['firstType'])                   
    second_Type.append(em['secondType'])                      
    third_Type.append(em['thirdType'])
    skill_Lables.append(em['skillLables'])
    position_Lables.append(em['positionLables'])
    industry_Lables.append(em['industryLables'])
    create_Time.append(em['createTime'])
    cityn.append(em['city'])
    districtn.append(em['district'])
    if em['businessZones']!=None:
        business_Zones.append(em['businessZones'])
    else:
        business_Zones.append(str(em['businessZones']))
    salaryn.append(em['salary'])
    work_Year.append(em['workYear'])
    job_Nature.append(em['jobNature'])
    educationn.append(em['education'])
    position_Advantage.append(em['positionAdvantage'])
    latituden.append(em['latitude'])
    longituden.append(em['longitude'])             

In [17]:
columns={'字段1':position,'字段2':company_name,'字段3':company_Size,'字段4':industry_Field,'字段5':finance_Stage,'字段6':companyLabel_List,'字段7':first_Type,'字段8':second_Type,'字段9':third_Type,'字段10':skill_Lables,'字段11':position_Lables,'字段12':industry_Lables,'字段13':create_Time,'字段14':cityn,'字段15':districtn,'字段16':business_Zones,'字段17':salaryn,'字段18':work_Year,'字段19':job_Nature,'字段20':educationn,'字段21':position_Advantage,'字段22':latituden,'字段23':longituden}


In [18]:
data=pd.DataFrame(columns)

In [19]:
data

,字段1,字段2,字段3,字段4,字段5,字段6,字段7,字段8,字段9,字段10,...,字段14,字段15,字段16,字段17,字段18,字段19,字段20,字段21,字段22,字段23
0,JAVA开发工程师,深圳金汇财富金融服务有限公司北京研发分公司,15-50人,"移动互联网,金融",不需要融资,"[带薪年假, 团队nice, 扁平管理, 通讯津贴]",开发|测试|运维类,后端开发,Java,[后端],...,北京,海淀区,"[公主坟, 万寿路]",8k-10k,应届毕业生,全职,本科,"节日福利,团建活动,年底双薪,年终奖",39.909371,116.311868
1,全球化广告-高级研发工程师,淘宝（中国）软件有限公司,2000人以上,电商,上市公司,"[年底双薪, 带薪年假, 午餐补助, 弹性工作]",开发|测试|运维类,后端开发,Java,"[平台, 分布式, J2EE]",...,北京,朝阳区,"[望京, 大山子]",30k-60k,3-5年,全职,硕士,核心业务，技术驱动业务增长，空间大机会多,39.999975,116.48678
2,java,网联清算有限公司,150-500人,金融,不需要融资,"[定期体检, 通讯津贴, 带薪年假, 午餐补助]",开发|测试|运维类,后端开发,后端开发,[],...,北京,西城区,[南礼士路],25k-50k,5-10年,全职,本科,金融业务,39.914947,116.353635
3,高级java开发工程师,网联清算有限公司,150-500人,金融,不需要融资,"[定期体检, 通讯津贴, 带薪年假, 午餐补助]",开发|测试|运维类,前端开发,WEB前端,"[Javascript, Web前端]",...,北京,西城区,[南礼士路],20k-40k,3-5年,全职,本科,发展前景,39.914947,116.353635
4,高级java开发工程师,海尔集团,2000人以上,"物联网,电商",上市公司,"[节日礼物, 带薪年假, 扁平管理, 五险一金]",开发|测试|运维类,后端开发,Java,[Java],...,北京,海淀区,"[中关村, 万泉河, 双榆树]",20k-40k,5-10年,全职,本科,五险一金 带薪年假 广阔成长空间,39.984622,116.31315
5,java高级开发工程师,海尔集团,2000人以上,"物联网,电商",上市公司,"[节日礼物, 带薪年假, 扁平管理, 五险一金]",开发|测试|运维类,后端开发,Java,"[分布式, 平台, 后端, 数据挖掘]",...,北京,海淀区,"[中关村, 万泉河, 双榆树]",26k-36k,5-10年,全职,本科,物联网平台前景好、薪资福利好、项目激励高,39.984622,116.31315
6,Java高级开发工程师,腾讯科技（深圳）有限公司,2000人以上,社交,上市公司,"[免费班车, 成长空间, 年度旅游, 岗位晋升]",开发|测试|运维类,后端开发,其他后端开发,"[分布式, Java]",...,北京,海淀区,"[北京大学, 中关村, 苏州街]",15k-25k,3-5年,全职,本科,"鹅厂好福利,大平台,大咖多",39.984207,116.307524
7,java工程师,北京百家互联科技有限公司,2000人以上,"移动互联网,教育",上市公司,"[节日礼物, 技能培训, 股票期权, 精英团队]",开发|测试|运维类,后端开发,Java,[],...,北京,海淀区,"[西二旗, 西北旺]",30k-55k,3-5年,全职,本科,六险一金，餐补，班车，节假日福利,40.044514,116.283842
8,高级java开发工程师,北京微聚未来科技有限公司,500-2000人,"移动互联网,金融",不需要融资,"[定期体检, 弹性工作, 免费班车, 七险一金]",开发|测试|运维类,后端开发,Java,"[Java, MySQL, 分布式]",...,北京,海淀区,"[西北旺, 上地, 马连洼]",20k-40k,3-5年,全职,本科,"牛人团队,带薪年假,16薪考核,快速成长",40.049953,116.284706
9,java开发工程师,北京微聚未来科技有限公司,500-2000人,"移动互联网,金融",不需要融资,"[定期体检, 弹性工作, 免费班车, 七险一金]",开发|测试|运维类,后端开发,Java,"[后端, Java, Javascript]",...,北京,海淀区,"[西北旺, 上地, 马连洼]",15k-30k,3-5年,全职,本科,"牛人团队,带薪年假,16薪考核,快速成长",40.049953,116.284706


# 爬取多页

for i in range的时候要慢慢爬，不能一下子就爬到很后面，这样也会被发现的。 可以这样理解：你翻页翻太快很明显就不是人为的，所以很容易被反爬。所以这时候我们要import time，就是增加延时。time.sleep(2),停两秒在翻页。

In [3]:
from requests_html import HTMLSession
from time import sleep
from fake_useragent import UserAgent
import json

爬虫过程中的反爬措施非常重要，其中设置随机 User-Agent 是一项重要的反爬措施，设置随机 UA 的方式有很多种，有的复杂有的简单，本文就对这些方法进行汇总，提供一种只需要一行代码的设置方式。

要想正确的获取请求信息，关键在于cookies，因为拉钩网的反爬机制在设置时头部信息中必须包含本页面所带的cookies信息，需要获取搜索结果那一页的cookie 以及header里面需要添加Accept、Referer、User-Agent。

In [5]:
#随机生成
#from fake_useragent import UserAgent
#ua = UserAgent()
#for i in range(10):
#    print(ua.random)

Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.17 (KHTML, like Gecko) Chrome/24.0.1312.60 Safari/537.17
Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.0 Safari/537.36
Mozilla/5.0 (Windows NT 6.2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1467.0 Safari/537.36
Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.67 Safari/537.36
Mozilla/5.0 (Windows NT 6.1; WOW64; rv:21.0) Gecko/20130401 Firefox/21.0
Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2049.0 Safari/537.36
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36
Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/534.55.3 (KHTML, li

In [35]:
position=[]
company_name=[]
company_Size=[]
industry_Field=[]
finance_Stage=[]
companyLabel_List=[]
first_Type=[]
second_Type=[]
third_Type=[]
skill_Lables=[]
position_Lables=[]
industry_Lables=[]
create_Time=[]
cityn=[]
districtn=[]
business_Zones=[]
salaryn=[]
work_Year=[]
job_Nature=[]
educationn=[]
position_Advantage=[]
linestaionn=[]
latituden=[]
longituden=[]

In [36]:
for i in range(5):
    #写入真实网址，不是网页上的网址，是在消息头那，别的浏览器是Headers
    session = HTMLSession()
    a = session.get(
    url='https://www.lagou.com/jobs/list_爬虫工程师?labelWords=&fromSearch=true&suginput=',
    headers={
        'useragent': UserAgent().random})
    sleep(5)
    url = 'https://www.lagou.com/jobs/positionAjax.json'
    ua = UserAgent()
    headers = {
    'user-agent': ua.random,
    'referer': 'https://www.lagou.com/jobs/list_%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90?city=%E5%85%A8%E5%9B%BD&cl=false&fromSearch=true&labelWords=&suginput=&labelWords=hot',
    'Origin': 'https://www.lagou.com',
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8'}
    
    data = {
           'first':'true',
           'pn':i,
           'kd':'java',
           'city': '北京',
           'needAddtionalResult': 'false'
            }
    r = session.post(
    url=url,
    headers=headers,
    data=data)
    content = r.content
    html = json.loads(content)
    htmlnew=html['content']
    

    for em in htmlnew['positionResult']['result']:
        position.append(em['positionName'])
        company_name.append(em['companyFullName'])
        company_Size.append(em['companySize'])
        industry_Field.append(em['industryField'])
        finance_Stage.append(em['financeStage'])
        companyLabel_List.append(em['companyLabelList'])
        first_Type.append(em['firstType'])                   
        second_Type.append(em['secondType'])                      
        third_Type.append(em['thirdType'])
        skill_Lables.append(em['skillLables'])
        position_Lables.append(em['positionLables'])
        industry_Lables.append(em['industryLables'])
        create_Time.append(em['createTime'])
        cityn.append(em['city'])
        districtn.append(em['district'])
        if em['businessZones']!=None:
            business_Zones.append(em['businessZones'])
        else:
            business_Zones.append(str(em['businessZones']))
        salaryn.append(em['salary'])
        work_Year.append(em['workYear'])
        job_Nature.append(em['jobNature'])
        educationn.append(em['education'])
        position_Advantage.append(em['positionAdvantage'])
        latituden.append(em['latitude'])
        longituden.append(em['longitude'])             
    

In [38]:
columns={'字段1':position,'字段2':company_name,'字段3':company_Size,'字段4':industry_Field,'字段5':finance_Stage,'字段6':companyLabel_List,'字段7':first_Type,'字段8':second_Type,'字段9':third_Type,'字段10':skill_Lables,'字段11':position_Lables,'字段12':industry_Lables,'字段13':create_Time,'字段14':cityn,'字段15':districtn,'字段16':business_Zones,'字段17':salaryn,'字段18':work_Year,'字段19':job_Nature,'字段20':educationn,'字段21':position_Advantage,'字段22':latituden,'字段23':longituden}


In [39]:
data=pd.DataFrame(columns)

In [40]:
data.shape

(75, 23)